In [12]:
import requests
import pandas as pd
import numpy as np

# Introduction

# Load dataset

The raw program data is available for download from the NY Philharmonic's official GitHub repository. The data is stored in `.json` format with a nested structure described at https://github.com/nyphilarchive/PerformanceHistory. The outermost key is `'programs'`.

In [237]:
url = 'https://raw.githubusercontent.com/nyphilarchive/PerformanceHistory/master/Programs/json/complete.json'

r = requests.get(url)
programs = r.json()['programs']

Each program consists of a list of works that is performed at multiple concerts. Using `pd.json_normalize`, we can load the details for each work and concert into separate data frames.

In [257]:
meta_cols = ['id', 'programID', 'orchestra', 'season']
df_concerts = pd.json_normalize(data=programs, record_path='concerts',
    meta=meta_cols)
df_works = pd.json_normalize(data=programs, record_path='works',
    meta=meta_cols)

In [258]:
df_concerts.head()

,eventType,Location,Venue,Date,Time,id,programID,orchestra,season
0,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
1,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-02-18T05:00:00Z,8:00PM,1118e84e-eb59-46cc-9119-d903375e65e6-0.1,5178,New York Philharmonic,1842-43
2,Special,"Manhattan, NY",Apollo Rooms,1843-04-07T05:00:00Z,8:00PM,08536612-27c3-437e-9b44-def21034b06c-0.1,10785,Musicians from the New York Philharmonic,1842-43
3,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-04-22T05:00:00Z,8:00PM,81a3b8de-1737-4c9e-9318-b839f7c7c4c0-0.1,5887,New York Philharmonic,1842-43
4,Subscription Season,"Manhattan, NY",Apollo Rooms,1843-11-18T05:00:00Z,None,09581bb7-8855-4965-b302-fc54cc669041-0.1,305,New York Philharmonic,1843-44


In [259]:
df_works.head()

,ID,composerName,workTitle,conductorName,soloists,movement,interval,movement._,movement.em,workTitle._,workTitle.em,id,programID,orchestra,season
0,52446*,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",[],NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
1,8834*4,"Weber, Carl Maria Von",OBERON,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
2,3642*,"Hummel, Johann","QUINTET, PIANO, D MINOR, OP. 74",NaN,"[{'soloistName': 'Scharfenberg, William', 'sol...",NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
3,0*,NaN,NaN,NaN,[],NaN,Intermission,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43
4,8834*3,"Weber, Carl Maria Von",OBERON,"Etienne, Denis G.",[],Overture,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43


We then outer merge the two datasets to construct a single `DataFrame`.

In [260]:
df = pd.merge(df_works, df_concerts, 'outer', on=meta_cols)

In [261]:
df.head()

,ID,composerName,workTitle,conductorName,soloists,movement,interval,movement._,movement.em,workTitle._,workTitle.em,id,programID,orchestra,season,eventType,Location,Venue,Date,Time
0,52446*,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",[],NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
1,8834*4,"Weber, Carl Maria Von",OBERON,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
2,3642*,"Hummel, Johann","QUINTET, PIANO, D MINOR, OP. 74",NaN,"[{'soloistName': 'Scharfenberg, William', 'sol...",NaN,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
3,0*,NaN,NaN,NaN,[],NaN,Intermission,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
4,8834*3,"Weber, Carl Maria Von",OBERON,"Etienne, Denis G.",[],Overture,NaN,NaN,NaN,NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM


# Clean columns

## `'.em'` and `'._`' columns

Check the proportion of missing values in each column:

In [262]:
df.isna().sum() / len(df)

ID               0.000138
composerName     0.148856
workTitle        0.148937
conductorName    0.201749
soloists         0.000138
movement         0.718664
interval         0.851282
movement._       0.998809
movement.em      0.997279
workTitle._      0.999919
workTitle.em     0.999919
id               0.000000
programID        0.000000
orchestra        0.000000
season           0.000000
eventType        0.000000
Location         0.000000
Venue            0.000000
Date             0.000000
Time             0.000000
dtype: float64

We notice a few strangely named columns with many missing values (e.g. `'movement.em'`). Can we move this information into the `'movement'` and `'workTitle'` columns? Upon examining some representative rows with information in these columns, I figured out that the `'.em'` column contains italicized text in the title of the work (see for example ID 8897*, where [Carmen is italicized in the program](https://archives.nyphil.org/index.php/artifact/7fa203d8-1167-4ec9-b2b0-11a45b02a4a7-0.1) (click "Show all")). This probably came from an `<em>` HTML tag.

In [263]:
cols = ['movement._', 'movement.em', 'workTitle._', 'workTitle.em']
df[df[cols].notna().any(axis=1)].sample(5, random_state=3)

,ID,composerName,workTitle,conductorName,soloists,movement,interval,movement._,movement.em,workTitle._,workTitle.em,id,programID,orchestra,season,eventType,Location,Venue,Date,Time
16817,8867*2,"Sibelius, Jean","LEMMINKAINEN SUITE (LEGENDS), OP. 22","Stransky, Josef",[],NaN,NaN,NaN,The Swan of Tuonela,NaN,NaN,0c97faa2-0a4f-4cec-8be6-089910a1a716-0.1,410,New York Philharmonic,1919-20,Tour,"Erie, PA",Park Opera House,1920-03-21T05:00:00Z,3:30PM
8651,6401*4,"Strauss, Richard","SONGS, FOUR (VIER LIEDER), OP. 33, (ORCHESTRA)","Mahler, Gustav","[{'soloistName': 'Bispham, David Scull', 'solo...",NaN,NaN,NaN,Pilgers Morgenlied,NaN,NaN,3ac3b545-d8c9-41b3-a677-90b569865267-0.1,5643,New York Philharmonic,1910-11,Subscription Season,"Brooklyn, NY",Brooklyn Academy of Music,1911-02-12T05:00:00Z,3:00PM
114010,8897*,"Bizet, Georges",NaN,NaN,"[{'soloistName': 'Mutter, Anne-Sophie', 'soloi...",NaN,NaN,NaN,NaN,CONCERT FANTASY ON FOR VIOLIN AND PIANO (ARR....,CARMEN,7fa203d8-1167-4ec9-b2b0-11a45b02a4a7-0.1,10648,NYP Presentation,2010-11,Artist in Residence / Soloist Recital,"Manhattan, NY",Avery Fisher Hall,2011-06-05T04:00:00Z,3:00PM
118318,5801*14,"Mendelssohn, Felix","MIDSUMMER NIGHT'S DREAM, OP. 61","Lewis, Courtney","[{'soloistName': 'Price, Christine', 'soloistI...",NaN,NaN,No. 13: Finale,"Through this house give glimmering light,",NaN,NaN,fc692230-87f4-40ff-a845-27ceff958407-0.1,12533,New York Philharmonic,2014-15,Young People's Concerts for Schools,"Manhattan, NY",Avery Fisher Hall,2015-03-04T05:00:00Z,10:30AM
36506,8867*2,"Sibelius, Jean","LEMMINKAINEN SUITE (LEGENDS), OP. 22","Reiner, Fritz",[],NaN,NaN,NaN,The Swan of Tuonela,NaN,NaN,6b6bf2ba-efb4-42ff-9040-22b90add5c9f-0.1,12190,Stadium-NY Philharmonic,1936-37,Stadium Concert,"Manhattan, NY",Lewisohn Stadium,1937-07-18T04:00:00Z,8:30PM


It won't be feasible to reconstruct the exact work title, but let's concatenate the strings in the two columns and impute it into the non-suffixed columns.
 
Is the `'movement'` column always empty when the other two have values present (and likewise for '`workTitle'`)? If so, the following code should print four `0`s.

In [264]:
for col in ['movement', 'workTitle']:
    for suffix in ['.em', '._']:
        print(df[df[col+suffix].notna()][col].notna().sum())

0
0
0
0


It is safe to consolidate these columns. Let's do that and drop the `.em` and `._` suffixed columns.

In [283]:
df2 = df.copy()

for col in ['movement', 'workTitle']:
    rows = df[col].isna()
    df2[col][rows] = df[col+'._'][rows] + ' ' +  df[col+'.em'][rows]
    df2.drop(columns=[col+'._', col+'.em'], inplace=True)

In [284]:
rows = df[col].isna()

In [285]:
df2[col][rows]

3         NaN
12        NaN
33        NaN
39        NaN
47        NaN
         ... 
123470    NaN
123471    NaN
123472    NaN
123473    NaN
123474    NaN
Name: workTitle, Length: 18390, dtype: object

In [286]:
df2.head()

,ID,composerName,workTitle,conductorName,soloists,movement,interval,id,programID,orchestra,season,eventType,Location,Venue,Date,Time
0,52446*,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",[],NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
1,8834*4,"Weber, Carl Maria Von",OBERON,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
2,3642*,"Hummel, Johann","QUINTET, PIANO, D MINOR, OP. 74",NaN,"[{'soloistName': 'Scharfenberg, William', 'sol...",NaN,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
3,0*,NaN,NaN,NaN,[],NaN,Intermission,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
4,8834*3,"Weber, Carl Maria Von",OBERON,"Etienne, Denis G.",[],Overture,NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM


## Interval

The 'interval' column only specifies intermissions. We should drop all these rows, as well as the column.

In [296]:
df2.interval.value_counts()

Intermission           18267
Intermission-Short        55
Intermission-Second       40
Intermission-Third         1
Name: interval, dtype: int64

In [298]:
# Double check that there is no useful information
df2[df2.interval=='Intermission'].composerName.any() 

False

In [304]:
df3 = df2[df2.interval.isna()].drop(columns='interval')

In [306]:
df3.head()

,ID,composerName,workTitle,conductorName,soloists,movement,id,programID,orchestra,season,eventType,Location,Venue,Date,Time
0,52446*,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",[],NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
1,8834*4,"Weber, Carl Maria Von",OBERON,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
2,3642*,"Hummel, Johann","QUINTET, PIANO, D MINOR, OP. 74",NaN,"[{'soloistName': 'Scharfenberg, William', 'sol...",NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
4,8834*3,"Weber, Carl Maria Von",OBERON,"Etienne, Denis G.",[],Overture,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM
5,8835*1,"Rossini, Gioachino",ARMIDA,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...",Duet,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07T05:00:00Z,8:00PM


In [411]:
df3.isna().sum()

ID                  17
composerName        17
workTitle           17
conductorName     6548
soloists            17
movement         70227
id                   0
programID            0
orchestra            0
season               0
eventType            0
Location             0
Venue                0
Date                 0
Time                 0
dtype: int64

## Date and Time
It may be useful to know the date and time of a performance, but the time zone doesn't really matter. Let's create a single DateTime column with `datetime` objects, throwing away the time zone and placing performances with no indicated time at midnight. We then drop the original date and time columns.

In [412]:
df3['Date'].sample()

27413    1928-11-11T05:00:00Z
Name: Date, dtype: object

In [413]:
# check that all dates split into two parts on the 'T'
(df3['Date'].str.split('T').str.len() == 2).all()

True

In [418]:
df4 = df3.copy()
df4['DateTime'] = pd.to_datetime(df3['Date'].str.split('T').str[0] \
                                 + ' ' + df3['Time'].str.replace('None', ''))
df4.drop(columns=['Date', 'Time'], inplace=True)
df4.head()

,ID,composerName,workTitle,conductorName,soloists,movement,id,programID,orchestra,season,eventType,Location,Venue,DateTime
0,52446*,"Beethoven, Ludwig van","SYMPHONY NO. 5 IN C MINOR, OP.67","Hill, Ureli Corelli",[],NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
1,8834*4,"Weber, Carl Maria Von",OBERON,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...","""Ozean, du Ungeheuer"" (Ocean, thou mighty mons...",00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
2,3642*,"Hummel, Johann","QUINTET, PIANO, D MINOR, OP. 74",NaN,"[{'soloistName': 'Scharfenberg, William', 'sol...",NaN,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
4,8834*3,"Weber, Carl Maria Von",OBERON,"Etienne, Denis G.",[],Overture,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00
5,8835*1,"Rossini, Gioachino",ARMIDA,"Timm, Henry C.","[{'soloistName': 'Otto, Antoinette', 'soloistI...",Duet,00646b9f-fec7-4ffb-9fb1-faae410bd9dc-0.1,3853,New York Philharmonic,1842-43,Subscription Season,"Manhattan, NY",Apollo Rooms,1842-12-07 20:00:00


In [419]:
df4.isna().sum()

ID                  17
composerName        17
workTitle           17
conductorName     6548
soloists            17
movement         70227
id                   0
programID            0
orchestra            0
season               0
eventType            0
Location             0
Venue                0
DateTime             0
dtype: int64

In [425]:
df4['orchestra'].value_counts()

New York Philharmonic                       77441
Stadium-NY Philharmonic                     11733
New York Symphony                            8814
Musicians from the New York Philharmonic     3075
Members of NY Philharmonic                   1540
NYP Presentation                             1129
NY Philharmonic Ensembles                     911
New/National Symphony Orchestra               407
Strike Orchestra (Philharmonic)                30
Shanghai Orchestra Academy                     18
Members of NY Symphony                         11
None                                            3
Name: orchestra, dtype: int64

# Ideas:
- Are works of some composers more likely to be programmed together?
- Are works from lesser-known composers programmed alongside those of better-known composers?
- Are on-tour concerts more or less adventurous than NYC concerts?
- Bring in subscribers dataset to get concert attendance by subscribers (but not full attendance?) https://archives.nyphil.org/index.php/open-data